In [2]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Eric Meinhardt / emeinhardt@ucsd.edu**

In [3]:
import numpy as np
myint = np.int8

In [4]:
from itertools import starmap, product, combinations, chain

In [5]:
from functools import reduce

In [6]:
from tqdm import tqdm

from joblib import Parallel, delayed, Memory

J = -1
BACKEND = 'multiprocessing'
# BACKEND = 'loky'
V = 10
PREFER = 'processes'
# PREFER = 'threads'

def par(gen_expr, j=None, backend=None, verbose=None, prefer=None):
    if j is None:
        j = J
    if backend is None:
        backend = BACKEND
    if verbose is None:
        verbose = V
    if prefer is None:
        prefer = PREFER
    return Parallel(n_jobs=j, backend=backend, verbose=verbose, prefer=prefer)(gen_expr)

def identity(x):
    return x

In [7]:
from random import choice

In [8]:
CAREFUL = False

In [9]:
import sparse

In [99]:
from scipy.special import binom#, comb

In [110]:
import os

# Read in (or make) object vectors

## Make

In [10]:
m = 5

In [11]:
max_num_objects = 2 ** m
max_num_objects

max_num_partial_fvs = (2 + 1) ** m
max_num_partial_fvs

32

243

In [12]:
def make_random_pfv():
    return np.random.randint(3, size=m, dtype=myint) - 1

In [13]:
max_num_objects
actual_num_objects = np.random.randint(max_num_objects)
# actual_num_objects = 40
actual_num_objects

assert actual_num_objects < max_num_objects

32

15

In [14]:
objects = tuple(set([tuple(np.random.randint(2, size=m)) for each in range(actual_num_objects)]))
objects = tuple(map(np.array, objects))
l = len(objects)

def zeroToMinusOne(u):
    return np.array([x if x == 1 else -1 for x in u])

objects = tuple([zeroToMinusOne(o) for o in objects])


actual_num_objects = len(objects)
actual_num_objects
objects

12

(array([ 1,  1, -1,  1, -1]),
 array([ 1,  1,  1, -1, -1]),
 array([-1, -1,  1,  1, -1]),
 array([ 1,  1,  1, -1,  1]),
 array([ 1,  1, -1,  1,  1]),
 array([ 1, -1,  1, -1,  1]),
 array([ 1, -1, -1,  1,  1]),
 array([ 1, -1,  1,  1, -1]),
 array([ 1, -1, -1, -1, -1]),
 array([ 1, -1, -1,  1, -1]),
 array([1, 1, 1, 1, 1]),
 array([-1,  1, -1, -1,  1]))

In [15]:
objectMap = np.array([objects[i] for i in range(l)])
objectMap.shape
objectMap
objectMap[0]

O = objectMap

(12, 5)

array([[ 1,  1, -1,  1, -1],
       [ 1,  1,  1, -1, -1],
       [-1, -1,  1,  1, -1],
       [ 1,  1,  1, -1,  1],
       [ 1,  1, -1,  1,  1],
       [ 1, -1,  1, -1,  1],
       [ 1, -1, -1,  1,  1],
       [ 1, -1,  1,  1, -1],
       [ 1, -1, -1, -1, -1],
       [ 1, -1, -1,  1, -1],
       [ 1,  1,  1,  1,  1],
       [-1,  1, -1, -1,  1]])

array([ 1,  1, -1,  1, -1])

## Read-in

In [16]:
# m = 5

In [17]:
# O = 

# Operations 

## Make generator vectors

In [18]:
def make_generator_vectors(num_features):
    basis_vectors = [np.zeros(num_features, dtype=myint) for each in range(num_features)]
    basis_vectors_neg = [np.zeros(num_features, dtype=myint) for each in range(num_features)]
    for i,v in enumerate(basis_vectors):
        v[i] = 1
    for i,v in enumerate(basis_vectors_neg):
        v[i] = -1
    generators = basis_vectors + basis_vectors_neg
    return generators

In [19]:
generators = make_generator_vectors(m)
generators

[array([1, 0, 0, 0, 0], dtype=int8),
 array([0, 1, 0, 0, 0], dtype=int8),
 array([0, 0, 1, 0, 0], dtype=int8),
 array([0, 0, 0, 1, 0], dtype=int8),
 array([0, 0, 0, 0, 1], dtype=int8),
 array([-1,  0,  0,  0,  0], dtype=int8),
 array([ 0, -1,  0,  0,  0], dtype=int8),
 array([ 0,  0, -1,  0,  0], dtype=int8),
 array([ 0,  0,  0, -1,  0], dtype=int8),
 array([ 0,  0,  0,  0, -1], dtype=int8)]

In [20]:
# max_num_objects = 2 ** m
# max_num_objects

# max_num_partial_fvs = (2 + 1) ** m
# max_num_partial_fvs

## Boilerplate

In [21]:
def wf_pfv(v):
    allowedValues = {-1,0,1}
    return all([x in allowedValues for x in v])

In [22]:
def wf_tfv(v):
    allowedValues = {-1,1}
    return all([x in allowedValues for x in v])

In [23]:
def uniquify(ndarray_iterable):
    tuples = [tuple(a) for a in ndarray_iterable]
    s = set(tuples)
    arrays = [np.array(t) for t in s]
    return arrays

## Agreement

In [24]:
def ag(x,y):
    '''
    Formula:
    (x == 0 or y == 0) or ((x != 0 and y != 0) and (x == y)), where T = 1 and F = 0
    
    Pattern:
    x = x ⟶ 1
    0 = _ ⟶ 1
    _ = 0 ⟶ 1
    _ = _ ⟶ 0
    '''
    if x == y:
        return True
    elif x == 0:
        return True
    elif y == 0:
        return True
    else:
        return False

In [25]:
def agree(u,v):
    '''
    Given two vectors u and v, returns a binary vector indicating,
    elementwise, whether u and v 'agree'.
    
    agree(u[i], v[i]) iff (u[i] == 0 or v[i] == 0) or (u[i] == v[i])
    '''
#     return np.array([True if (u[i] == 0 or v[i] == 0) or (u[i] == v[i]) else False 
#                      for i in range(len(u))])
    return np.array([1 if (u[i] == 0 or v[i] == 0) or (u[i] == v[i]) else 0 
                     for i in range(len(u))], dtype=myint)

In [26]:
def agree_(u,v):
    '''
    Given two vectors u and v, return 1 iff u and v agree at all indices
    and 0 otherwise.
    '''
    ag = agree(u,v)
    return int(ag.all())

In [27]:
def agree_mat(A,B):
    '''
    Given two matrices A::(n,m) and B::(n,m), 
    return C::(n,1) where 
    C[i] = 1 iff A[i] and B[i] agree at all indices
    and 0 otherwise.
    '''
    # (x == 0 or y == 0) or ((x != 0 and y != 0) and (x == y))
    A_unspecified = A == 0
    B_unspecified = B == 0
    A_or_B_unspecified = A_unspecified | B_unspecified
    
    A_specified = A != 0
    B_specified = B != 0
    A_and_B_specified = A_specified & B_specified
    A_equal_B = np.equal(A,B)
    A_B_both_specified_and_equal = A_and_B_specified & A_equal_B

    ag = A_or_B_unspecified | A_B_both_specified_and_equal
#     return ag
    result = np.prod(ag, axis=-1, dtype=myint)
    return result

In [28]:
def make_agreeing_vector_pair(pred=None):
    u = make_random_pfv()
    v = make_random_pfv()
    if pred is None:
        while not agree_(u,v):
            u = make_random_pfv()
            v = make_random_pfv()
        return u,v
    while not agree_(u,v) and not pred(u,v):
        u = make_random_pfv()
        v = make_random_pfv()
    return u,v

In [29]:
num_test_pairs = int(1e5)
random_vector_pairs = [(make_random_pfv(), make_random_pfv()) for each in range(num_test_pairs)]
len(random_vector_pairs)

100000

In [30]:
num_test_pairs = int(1e5)
agreeing_vector_pairs = [make_agreeing_vector_pair() for each in range(num_test_pairs)]
len(agreeing_vector_pairs)

100000

In [31]:
first = lambda seq: seq[0]
second = lambda seq: seq[1]

stack_a, stack_b = list(map(first, random_vector_pairs)), list(map(second, random_vector_pairs))
random_pair_stack_a, random_pair_stack_b = np.array(stack_a), np.array(stack_b)
random_pair_stack_a.dtype
random_pair_stack_b.dtype

dtype('int8')

dtype('int8')

In [32]:
stack_a, stack_b = list(map(first, agreeing_vector_pairs)), list(map(second, agreeing_vector_pairs))
agreeing_pair_stack_a, agreeing_pair_stack_b = np.array(stack_a), np.array(stack_b)
agreeing_pair_stack_a.dtype
agreeing_pair_stack_b.dtype

dtype('int8')

dtype('int8')

In [32]:
%%timeit

list(starmap(agree_, random_vector_pairs));

1.47 s ± 2.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%%timeit

agree_mat(random_pair_stack_a, random_pair_stack_b)

2.73 ms ± 4.81 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [33]:
np.array_equal(agree_mat(random_pair_stack_a, random_pair_stack_b), 
               list(starmap(agree_, random_vector_pairs)))

True

In [34]:
n = num_test_pairs
for i in range(n):
    u = random_pair_stack_a[i]
    v = random_pair_stack_b[i]
    assert agree_(u,v) == agree_mat(u,v), '{0}, {1} -> {2} vs. {3}'.format(u,v, agree_(u,v), agree_mat(u,v, True))

In [35]:
agreement = agree_mat

## Union

In [36]:
XYs = tuple(product((-1,0,1), (-1,0,1)))
XYs

def cup(x,y):
    '''
    Formula:
    x or y, where 1 = T, -1 = T, 0 = F
    
    Algebra:
    0 is the identity ∀x ∈ {-1,0,+1}
    x is its own identity ∀x ∈ {-1,0,+1}
    (-1 and +1 are mutual inverses, but this case shouldn't occur when agree(x,y) holds)
    
    Pattern:
    x ∪ x = x
    
    0 ∪ y = y
    x ∪ 0 = x
    
    _ ∪ _ = 0  \\ <- shouldn't occur in two pfvs that agree
    '''
    if x == 0:  #if x is unspecified, return y
        return y
    elif y == 0: #if y is unspecified, return x
        return x
    elif x == y: #if both are specified and the same, return their common value
        return x
    else: #otherwise return 0
        return 0

for x,y in XYs:
    ((x,y), cup(x,y))

((-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 0), (0, 1), (1, -1), (1, 0), (1, 1))

((-1, -1), -1)

((-1, 0), -1)

((-1, 1), 0)

((0, -1), -1)

((0, 0), 0)

((0, 1), 1)

((1, -1), 0)

((1, 0), 1)

((1, 1), 1)

In [37]:
def union(u, v):
    if CAREFUL:
        assert agree_(u,v)
    return np.sign(u + v)

## Intersection

In [38]:
XYs = tuple(product((-1,0,1), (-1,0,1)))
XYs 
    
def cap(x,y):
    '''
    Algebra:
    0 is the annihilating element ∀x ∈ {-1,0,+1}
    x is its own identity ∀x ∈ {-1,0,+1}
    -1 and +1 annihilate each other
    
    Pattern:
    x ∩ x = x
    
    0 ∩ _ = 0
    _ ∩ 0 = 0
    
    _ ∩ _ = 0
    '''
    if x == 0: #if x is unspecified, return 0
        return 0
    elif y == 0: #if y is unspecified, return 0
        return 0
    elif x == y: #if both are specified and the same, return their common value
        return x
    else: #otherwise return 0
        return 0

def foo(x,y):
    return np.sign( (x == y) * (x + y) )

# def bar(x,y):
#     return (x == y) * (x + y) * 0.5

# def baz(x,y):
#     return (x == y) * int((x + y) / 2)

for x,y in XYs:
#     ((x,y), cap(x,y))
#     ((x,y), cap(x,y), foo(x,y), bar(x,y), baz(x,y))
    ((x,y), cap(x,y), foo(x,y))

((-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 0), (0, 1), (1, -1), (1, 0), (1, 1))

((-1, -1), -1, -1)

((-1, 0), 0, 0)

((-1, 1), 0, 0)

((0, -1), 0, 0)

((0, 0), 0, 0)

((0, 1), 0, 0)

((1, -1), 0, 0)

((1, 0), 0, 0)

((1, 1), 1, 1)

In [39]:
def intersection(u, v):
    return np.sign(  np.equal(u, v) * (u + v) )

## Extension

In [40]:
def getIndex(o, O):
    matches = [i for i,v in enumerate(O) if np.array_equal(v,o)]
    if len(matches) == 0:
        return -1
    if CAREFUL:
        assert len(matches) == 1
    return matches[0]

In [41]:
def makeExtensionVector(positive_Indices, O):
    return np.array([1 if i in positive_Indices else 0 for i in np.arange(O.shape[0])], dtype=myint)

In [42]:
def extension(v, O, asIndexVector=True):
    '''
    The extension of a partial feature vector v is the set of object vectors
    (= fully specified, or 'total' feature vectors) that 'agree' with it.
    '''
    matches = tuple([o for o in O if agree_(v,o)])
#     matches = tuple([o for o in objects if agree(v,o).all()])
#     matches = np.array([1.0 if np.linalg.norm(agree(v,o), 1) == num_features else 0.0 for o in objects])
    if asIndexVector:
        return makeExtensionVector([getIndex(o, O) for o in matches], O)
    return matches

In [43]:
def ramp(M):
    return np.heaviside(M-1, 1).astype(myint)

def primed(p):
    mag_p = np.sum(np.abs(p))
    return p / mag_p

def extension_alt3(s, O):
    if np.array_equal(s, np.zeros(s.shape)):
        return np.ones((l,), dtype=myint)
    p = s
#     mag_p = np.sum(np.abs(p))
#     p_prime = p / mag_p
    return ramp( np.dot(O, primed(p)) )

In [44]:
def extension_(pfv, O):
    return agree_mat(pfv, O)

In [45]:
num_test_pairs = int(1e5)
random_vectors = [make_random_pfv() for each in tqdm(range(num_test_pairs))]
len(random_vectors)

100%|██████████| 100000/100000 [00:00<00:00, 275910.56it/s]


100000

In [47]:
%%timeit

list(map(lambda v: extension(v, O), random_vectors))

19.8 s ± 56.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
%%timeit

list(map(lambda v: extension_alt3(v, O), random_vectors))

1.46 s ± 2.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [49]:
%%timeit

list(map(lambda v: extension_(v,O), random_vectors))

1.24 s ± 12.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [46]:
interpretation = extension_

## Entailed pfvs

In [47]:
def specifiable_zero_indices(p, ext_p):
    '''
    Given p and A::(n,m) = ⟦p⟧:
    
    If p_j = 0 and ∀i A_{i,j} = k≠0, then
    p_j is unspecified (i.e. p_j = 0) but 
    can be set to k and yield a co-extensive 
    and more specific pfv p'. (NB: p' entails 
    p.)
    
    This function returns a list of (index, value) pairs
    indicating the set of 0-valued indices of p that can 
    be specified, plus what the common value at that index is.
    
    Correctly specifying any one or any combination
    of the indices in this list of indices will result
    in a more specific vector than p that is coextensive.
    
    From this list, you can construct (or count) all of the
    more specified pfvs that are coextensive with p.
    '''
    A = ext_p
    n = A.shape[0]
    if n == 0:
        return set()
    n_opp = -1.0 * n
#     zeros = np.nonzero(p)[0]
    zero_indices = np.array(tuple(  set(range(len(p))) - set(np.nonzero(p)[0])  ), dtype=myint)
    specifiable_indices = set()
    for j in zero_indices:
        j_col_sum = np.sum(A[:,j])
        if j_col_sum == n:
            specifiable_indices.add((j, 1))
        if j_col_sum == n_opp:
            specifiable_indices.add((j, -1))
    return specifiable_indices

def specify(p, specs):
    '''
    Given a partial feature vector p and a set of
        (index i, non-zero value v)
    pairs where p_i ≠ 0, returns a more specific p'
    where p'_i = v as indicated by spec.
    '''
    p_prime = p.copy()
    for i,v in specs:
        p_prime[i] = v
    return p_prime

def entailed_pfvs(p, O, no_total_fvs = True):
    '''
    Given a partial feature vector p and a set of objects
    (total feature vectors) O, this returns the set of
    partial feature vectors that are strictly more specific
    than p that have the same extension in O.
    '''
    x_p = np.array(extension(p, O, False))
    specifiable_indices = specifiable_zero_indices(p, x_p)
    num_specifiable_indices = len(specifiable_indices)
    specifications = {tuple(combinations(specifiable_indices, r) )
                      for r in range(1, num_specifiable_indices+1)}
    entailed_vectors = np.array([specify(p, spec)
                                 for r_level in specifications 
                                 for spec in r_level], dtype=myint)
    if not no_total_fvs:
        return entailed_vectors
    entailed_pfvs = np.array([v for v in entailed_vectors
                              if len(v.nonzero()[0]) < m])
    return entailed_pfvs

# Generation of $S_i$: all pfvs with exactly $i$ specified values

In [52]:
# from functools import reduce

In [48]:
def grand_union(pfvs):
    return reduce(union, pfvs)

In [49]:
def one_hot_stack(indices):
#     n_values = np.max(indices) + 1
#     n_values = num_features
    n_values = m
    return np.eye(n_values,dtype=myint)[indices] 

In [50]:
def indexChoicesToComponentOptions(index_choices):
    indices = list(index_choices)
    one_hots = one_hot_stack(indices)
#     component_options = tuple([(v, -1 * v) for v in one_hots])
    component_options = ((v, -1 * v) for v in one_hots)
    return component_options

def componentOptionsToChoices(component_options):
#     choice_combinations = tuple(product(*component_options))
    choice_combinations = product(*component_options)
#     return tuple(starmap(union,
#                          choice_combinations))
#     return tuple(map(grand_union,
#                      choice_combinations))
    return map(grand_union, choice_combinations)

def make_Si_naive(i):
    index_choices = combinations(range(m), i)
    componentOptions = (indexChoicesToComponentOptions(c) for c in index_choices)
    componentChoices = (componentOptionsToChoices(o) for o in componentOptions)
#     choices_flattened = reduce(lambda a,b: a + b, componentChoices)
    choices_flattened = tuple(reduce(lambda a,b: chain.from_iterable([a,b]), componentChoices))
    return np.array(choices_flattened)

In [51]:
construct_Si = make_Si_naive

In [52]:
# calculate_Xi = interpretation

#FIXME this can/should be parallelized and memory mapped
def calculate_Xi_naive(Si, O):
    return np.array([interpretation(p, O) for p in Si], dtype=myint)

In [53]:
def heaviside(x):
    return np.array(1 * (x >= 0))

def extension_multi_bool(p_mat,V):
    """
    Compute a boolean vector that represents the extension of p in V
    
    Inputs:
        p_mat - a matrix of shape (M,num_p) with elements from {-1,0,1}.  The matrix of partially specified
            feature vectors, containing num_p vectors
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        extension - a matrix of shape (L,num_p) with elements from {1,0}.  extension[l,i]=1 iff V[l,:] is 
            in the extension of p_mat[:,i]
    """
    K_vec = np.sum(abs(p_mat),axis=0) #shape is (num_p,)
    E = np.dot(V,p_mat) #shape is (L,num_p)
    return heaviside(E-K_vec[np.newaxis,:])

# calculate_Xi = extension_multi_bool

def calculate_Xi(p_mat, V):
    """
    Compute a boolean vector that represents the extension of p in V
    
    Inputs:
        p_mat - a matrix of shape (num_p, M) with elements from {-1,0,1}.  The matrix of partially specified
            feature vectors, containing num_p vectors
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        extension - a matrix of shape (L,num_p) with elements from {1,0}.  extension[l,i]=1 iff V[l,:] is 
            in the extension of p_mat[:,i]
    """
    p_mat_prime = p_mat.T
    K_vec = np.sum(abs(p_mat_prime),axis=0) #shape is (num_p,)
    E = np.dot(V,p_mat_prime) #shape is (L,num_p)
    result = heaviside(E-K_vec[np.newaxis,:]).T
    
#     K_vec_prime = np.sum(abs(p_mat), axis=1)
# #     assert np.array_equal(K_vec_prime, K_vec.T)
#     E_prime = np.dot(p_mat, V.T)
# #     assert np.array_equal(E_prime, E.T)
#     result_prime = heaviside(E_prime-K_vec_prime[:,np.newaxis])
#     assert result_prime.shape == result.shape, '{0} vs. {1}'.format(result_prime.shape, result.shape)
#     assert np.array_equal(result_prime, result.T)    
    return result#_prime


In [54]:
O.shape

(12, 5)

In [55]:
m

5

In [56]:
S3 = construct_Si(3)
S3.shape

(80, 5)

In [57]:
S3.T.shape

(5, 80)

In [58]:
O.shape

(12, 5)

In [59]:
%%timeit

calculate_Xi_naive(S3, O)

1.11 ms ± 5.43 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [148]:
%%timeit

calculate_Xi(S3, O)

18.2 µs ± 58.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [60]:
calculate_Xi_naive(S3, O).shape
calculate_Xi_naive(S3, O)

(80, 12)

array([[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0,

In [61]:
calculate_Xi(S3, O).shape
calculate_Xi(S3, O)

(80, 12)

array([[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0,

# Generate $\overline{S}_i$, $\overline{X}_i$ by removing vectors with empty extension in $S_i$ + their empty extension in $X_i$

In [58]:
EMPTY = np.zeros((l,), dtype=myint)

In [73]:
#FIXME this can/should be parallelized and memory mapped
def make_Si_bar_naive(Si, Xi):
    return np.array([v for i,v in enumerate(Si) 
#                      if not empty_extension(Xj[i])])
                     if not np.array_equal(EMPTY, Xi[i])])

In [71]:
#FIXME this can/should be parallelized and memory mapped
def make_Si_bar_Xi_bar_naive(Si, Xi):
    non_empty_indices = np.array([i for i,v in enumerate(Si)
                                  if not np.array_equal(EMPTY, Xi[i])])
    Si_bar = np.array([Si[i] for i in non_empty_indices])
    Xi_bar = np.array([Xi[i] for i in non_empty_indices])
    return Si_bar, Xi_bar

In [75]:
def make_Si_bar_Xi_bar_alt(Si, Xi):
#     non_empty_extension_row_indices = np.array([i for i,v in enumerate(Si)
#                                                 if np.sum(Xi[i]) != 0])
    Xi_sums = np.sum(Xi, axis=1) #shape is (Si.shape[0],)
    non_empty_extension_row_indices = Xi_sums.nonzero()[0]
    
    Si_bar = Si[non_empty_extension_row_indices,:]
    Xi_bar = Xi[non_empty_extension_row_indices,:]
    return Si_bar, Xi_bar

In [76]:
construct_Si_bar = make_Si_bar_naive
construct_Si_Xi_bar = make_Si_bar_Xi_bar_alt

In [172]:
S3.shape

(80, 5)

In [110]:
X3 = calculate_Xi_naive(S3, O)

In [159]:
X3.shape

(80, 9)

In [163]:
l, m

(9, 5)

In [162]:
O.shape

(9, 5)

In [169]:
np.sum(X3, axis=1).shape

(80,)

In [175]:
np.array([np.sum(X3[i]) for i,v in enumerate(S3)]).shape
np.array([np.sum(X3[i]) for i,v in enumerate(S3)]).nonzero()[0]

(80,)

array([ 0,  2,  3,  4,  5,  6,  7,  8, 11, 12, 13, 14, 15, 16, 18, 19, 20,
       21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 37, 38, 39, 40,
       42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 61,
       62, 64, 65, 66, 67, 69, 70, 71, 72, 73, 75, 77, 78])

In [168]:
non_empty_row_indices = np.array([i for i,v in enumerate(S3)
                                  if np.sum(X3[i]) != 0])
non_empty_row_indices.shape

(64,)

In [157]:
np.array_equal( make_Si_bar_Xi_bar_naive(S3, X3)[0], make_Si_bar_Xi_bar_alt(S3, X3)[0] )

True

In [158]:
np.array_equal( make_Si_bar_Xi_bar_naive(S3, X3)[1], make_Si_bar_Xi_bar_alt(S3, X3)[1] )

True

In [177]:
%%timeit

make_Si_bar_Xi_bar_naive(S3, X3)

286 µs ± 1.74 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [178]:
%%timeit

make_Si_bar_Xi_bar_alt(S3, X3)

9.55 µs ± 55.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Convert $\overline{X}_i$ to a sparse representation

In [62]:
# import sparse

In [78]:
def density(a):
    num_cells = reduce(lambda x,y: x * y, a.shape)
    d = len(np.nonzero(a)[0]) / num_cells
    return d

def sparsity(a):
    return 1 - density(a)

In [79]:
def to_sparse(v):
    return sparse.COO(v)

# Local processing pipeline to generate $\overline{S}_i$ and (dense) $\overline{X}_i$, $\forall i$ 

In [68]:
def construct_Si_bar_Xi_bar(i, O):
    Si = construct_Si(i)
    Xi = calculate_Xi(Si, O)
    Si_bar, Xi_bar = construct_Si_Xi_bar(Si, Xi)
#     Si_bar = construct_Si_bar(Si, Xi)
#     del Si
#     del Xi
    #FIXME you shouldn't have to recalculate the extensions of everything in Si_bar!
#     Xi_bar = calculate_Xi(Si_bar, O)
    return Si_bar, Xi_bar #these (or at least Xi_bar) should be sparse (and memory mapped) representations

In [63]:
construct_Si(1)

array([[ 1,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0],
       [ 0, -1,  0,  0,  0],
       [ 0,  0,  1,  0,  0],
       [ 0,  0, -1,  0,  0],
       [ 0,  0,  0,  1,  0],
       [ 0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  1],
       [ 0,  0,  0,  0, -1]], dtype=int8)

In [80]:
S1_bar, X1_bar = construct_Si_bar_Xi_bar(1, O)
sparsity(S1_bar)
sparsity(X1_bar)

0.8

0.5

In [81]:
S1_bar

array([[ 1,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0],
       [ 0, -1,  0,  0,  0],
       [ 0,  0,  1,  0,  0],
       [ 0,  0, -1,  0,  0],
       [ 0,  0,  0,  1,  0],
       [ 0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  1],
       [ 0,  0,  0,  0, -1]], dtype=int8)

In [82]:
S2_bar, X2_bar = construct_Si_bar_Xi_bar(2, O)
sparsity(S2_bar)
sparsity(X2_bar)

0.6

0.75

In [83]:
S3_bar, X3_bar = construct_Si_bar_Xi_bar(3, O)
sparsity(S3_bar)
sparsity(X3_bar)

0.4

0.8529411764705882

In [84]:
S4_bar, X4_bar = construct_Si_bar_Xi_bar(4, O)
sparsity(S4_bar)
sparsity(X4_bar)

0.19999999999999996

0.8979591836734694

In [85]:
S5_bar, X5_bar = construct_Si_bar_Xi_bar(5, O)
sparsity(S5_bar)
sparsity(X5_bar)

0.0

0.9166666666666666

# Memory-mapping

In [113]:
X5_bar_sparse = to_sparse(X5_bar)

In [114]:
X5_bar_sparse.nbytes

288

In [129]:
def construct_Si_bar_Xi_bar_mmap(i, O):
    Si_fn = 'S{0}.dat'.format(i)
    n_pfvs = int(binom(m, i) * (2 ** i))
    n_features = m
    Si_shape = (n_pfvs, n_features)
    print('Si_shape: {0}'.format(Si_shape))
    Si = np.memmap(Si_fn, dtype=myint, mode='w+', shape = Si_shape)
#     Si = construct_Si(i)
    Si[:] = construct_Si(i)
    print('Finished writing S{0} to disk as {1} w/ {2} GB'.format(i, Si_fn, Si.nbytes / 1e9))
    
    Xi_fn = 'X{0}.dat'.format(i)
    n_objects = l
    Xi_shape = (n_pfvs, n_objects)
    Xi = np.memmap(Xi_fn, dtype=myint, mode='w+', shape = Xi_shape)
    Xi[:] = calculate_Xi(Si, O)
    print('Finished writing X{0} to disk as {1} w/ {2} GB'.format(i, Xi_fn, Xi.nbytes / 1e9))
    
    Xi_sums = np.sum(Xi, axis=1) #shape is (Si.shape[0],)
    non_empty_extension_row_indices = Xi_sums.nonzero()[0]
    num_ne_pfvs = len(non_empty_extension_row_indices)
    print('Finished identifying {0} pfvs with non-empty extensions.'.format(num_ne_pfvs))
    
    Si_bar_fn = 'S{0}_bar.dat'.format(i)
    Si_bar_shape = (num_ne_pfvs, n_features)
    Si_bar = np.memmap(Si_bar_fn, dtype=myint, mode='w+', shape = Si_bar_shape)
    Si_bar[:] = Si[non_empty_extension_row_indices,:]
    print('Finished writing S{0}_bar to disk as {1} w/ {2} GB'.format(i, Si_bar_fn, Si_bar.nbytes / 1e9))
    print('Sparsity of S{0}_bar: {1}'.format(i, sparsity(Si_bar)))
    print('Deleting S{0} from disk, freeing {1} GB'.format(i, Si.nbytes / 1e9))
    os.remove(Si_fn)
    print('S{0} deleted'.format(i))
    
    Xi_bar_fn = 'X{0}_bar.dat'.format(i)
    Xi_bar_shape = (num_ne_pfvs, n_objects)
    Xi_bar = np.memmap(Xi_bar_fn, dtype=myint, mode='w+', shape = Xi_bar_shape)
    Xi_bar[:] = Xi[non_empty_extension_row_indices,:]
    print('Finished writing X{0}_bar to disk as {1} w/ {2} GB'.format(i, Xi_bar_fn, Xi_bar.nbytes / 1e9))
    print('Deleting X{0} from disk, freeing {1} GB'.format(i, Xi.nbytes / 1e9))
    os.remove(Xi_fn)
    print('X{0} deleted'.format(i))
    
    print('Sparsity of X{0}_bar: {1}'.format(i, sparsity(Xi_bar)))
    print('Creating sparse version of X{0}_bar'.format(i))
    Xi_bar_sparse_fn = 'X{0}_bar.sparse'.format(i)
    Xi_bar_sparse = to_sparse(Xi_bar)
    print('Saving as {0}'.format(Xi_bar_sparse_fn))
    sparse.save_npz(Xi_bar_sparse_fn, Xi_bar_sparse)
    print('Saved, using {0} GB'.format(Xi_bar_sparse.nbytes / 1e9))
    print('Deleting {0}, saving {1} GB'.format(Xi_bar_fn, Xi_bar.nbytes / 1e9))
    
    return Si_bar, Xi_bar_sparse #these (or at least Xi_bar) should be sparse (and memory mapped) representations

In [127]:
m

5

In [131]:
for i in range(1,m+1):
    print('i = {0}'.format(i))
    construct_Si_bar_Xi_bar_mmap(i, O)
    print(' ')

i = 1
Si_shape: (10, 5)
Finished writing S1 to disk as S1.dat w/ 5e-08 GB
Finished writing X1 to disk as X1.dat w/ 1.2e-07 GB
Finished identifying 10 pfvs with non-empty extensions.
Finished writing S1_bar to disk as S1_bar.dat w/ 5e-08 GB
Sparsity of S1_bar: 0.8
Deleting S1 from disk, freeing 5e-08 GB
S1 deleted
Finished writing X1_bar to disk as X1_bar.dat w/ 1.2e-07 GB
Deleting X1 from disk, freeing 1.2e-07 GB
X1 deleted
Sparsity of X1_bar: 0.5
Creating sparse version of X1_bar
Saving as X1_bar.sparse
Saved, using 1.02e-06 GB
Deleting X1_bar.dat, saving 1.2e-07 GB


(memmap([[ 1,  0,  0,  0,  0],
         [-1,  0,  0,  0,  0],
         [ 0,  1,  0,  0,  0],
         [ 0, -1,  0,  0,  0],
         [ 0,  0,  1,  0,  0],
         [ 0,  0, -1,  0,  0],
         [ 0,  0,  0,  1,  0],
         [ 0,  0,  0, -1,  0],
         [ 0,  0,  0,  0,  1],
         [ 0,  0,  0,  0, -1]], dtype=int8),
 <COO: shape=(10, 12), dtype=int8, nnz=60, fill_value=0>)

 
i = 2
Si_shape: (40, 5)
Finished writing S2 to disk as S2.dat w/ 2e-07 GB
Finished writing X2 to disk as X2.dat w/ 4.8e-07 GB
Finished identifying 40 pfvs with non-empty extensions.
Finished writing S2_bar to disk as S2_bar.dat w/ 2e-07 GB
Sparsity of S2_bar: 0.6
Deleting S2 from disk, freeing 2e-07 GB
S2 deleted
Finished writing X2_bar to disk as X2_bar.dat w/ 4.8e-07 GB
Deleting X2 from disk, freeing 4.8e-07 GB
X2 deleted
Sparsity of X2_bar: 0.75
Creating sparse version of X2_bar
Saving as X2_bar.sparse
Saved, using 2.04e-06 GB
Deleting X2_bar.dat, saving 4.8e-07 GB


(memmap([[ 1,  1,  0,  0,  0],
         [ 1, -1,  0,  0,  0],
         [-1,  1,  0,  0,  0],
         [-1, -1,  0,  0,  0],
         [ 1,  0,  1,  0,  0],
         [ 1,  0, -1,  0,  0],
         [-1,  0,  1,  0,  0],
         [-1,  0, -1,  0,  0],
         [ 1,  0,  0,  1,  0],
         [ 1,  0,  0, -1,  0],
         [-1,  0,  0,  1,  0],
         [-1,  0,  0, -1,  0],
         [ 1,  0,  0,  0,  1],
         [ 1,  0,  0,  0, -1],
         [-1,  0,  0,  0,  1],
         [-1,  0,  0,  0, -1],
         [ 0,  1,  1,  0,  0],
         [ 0,  1, -1,  0,  0],
         [ 0, -1,  1,  0,  0],
         [ 0, -1, -1,  0,  0],
         [ 0,  1,  0,  1,  0],
         [ 0,  1,  0, -1,  0],
         [ 0, -1,  0,  1,  0],
         [ 0, -1,  0, -1,  0],
         [ 0,  1,  0,  0,  1],
         [ 0,  1,  0,  0, -1],
         [ 0, -1,  0,  0,  1],
         [ 0, -1,  0,  0, -1],
         [ 0,  0,  1,  1,  0],
         [ 0,  0,  1, -1,  0],
         [ 0,  0, -1,  1,  0],
         [ 0,  0, -1, -1,  0],
        

 
i = 3
Si_shape: (80, 5)
Finished writing S3 to disk as S3.dat w/ 4e-07 GB
Finished writing X3 to disk as X3.dat w/ 9.6e-07 GB
Finished identifying 68 pfvs with non-empty extensions.
Finished writing S3_bar to disk as S3_bar.dat w/ 3.4e-07 GB
Sparsity of S3_bar: 0.4
Deleting S3 from disk, freeing 4e-07 GB
S3 deleted
Finished writing X3_bar to disk as X3_bar.dat w/ 8.16e-07 GB
Deleting X3 from disk, freeing 9.6e-07 GB
X3 deleted
Sparsity of X3_bar: 0.8529411764705882
Creating sparse version of X3_bar
Saving as X3_bar.sparse
Saved, using 2.04e-06 GB
Deleting X3_bar.dat, saving 8.16e-07 GB


(memmap([[ 1,  1,  1,  0,  0],
         [ 1,  1, -1,  0,  0],
         [ 1, -1,  1,  0,  0],
         [ 1, -1, -1,  0,  0],
         [-1,  1, -1,  0,  0],
         [-1, -1,  1,  0,  0],
         [ 1,  1,  0,  1,  0],
         [ 1,  1,  0, -1,  0],
         [ 1, -1,  0,  1,  0],
         [ 1, -1,  0, -1,  0],
         [-1,  1,  0, -1,  0],
         [-1, -1,  0,  1,  0],
         [ 1,  1,  0,  0,  1],
         [ 1,  1,  0,  0, -1],
         [ 1, -1,  0,  0,  1],
         [ 1, -1,  0,  0, -1],
         [-1,  1,  0,  0,  1],
         [-1, -1,  0,  0, -1],
         [ 1,  0,  1,  1,  0],
         [ 1,  0,  1, -1,  0],
         [ 1,  0, -1,  1,  0],
         [ 1,  0, -1, -1,  0],
         [-1,  0,  1,  1,  0],
         [-1,  0, -1, -1,  0],
         [ 1,  0,  1,  0,  1],
         [ 1,  0,  1,  0, -1],
         [ 1,  0, -1,  0,  1],
         [ 1,  0, -1,  0, -1],
         [-1,  0,  1,  0, -1],
         [-1,  0, -1,  0,  1],
         [ 1,  0,  0,  1,  1],
         [ 1,  0,  0,  1, -1],
        

 
i = 4
Si_shape: (80, 5)
Finished writing S4 to disk as S4.dat w/ 4e-07 GB
Finished writing X4 to disk as X4.dat w/ 9.6e-07 GB
Finished identifying 49 pfvs with non-empty extensions.
Finished writing S4_bar to disk as S4_bar.dat w/ 2.45e-07 GB
Sparsity of S4_bar: 0.19999999999999996
Deleting S4 from disk, freeing 4e-07 GB
S4 deleted
Finished writing X4_bar to disk as X4_bar.dat w/ 5.88e-07 GB
Deleting X4 from disk, freeing 9.6e-07 GB
X4 deleted
Sparsity of X4_bar: 0.8979591836734694
Creating sparse version of X4_bar
Saving as X4_bar.sparse
Saved, using 1.02e-06 GB
Deleting X4_bar.dat, saving 5.88e-07 GB


(memmap([[ 1,  1,  1,  1,  0],
         [ 1,  1,  1, -1,  0],
         [ 1,  1, -1,  1,  0],
         [ 1, -1,  1,  1,  0],
         [ 1, -1,  1, -1,  0],
         [ 1, -1, -1,  1,  0],
         [ 1, -1, -1, -1,  0],
         [-1,  1, -1, -1,  0],
         [-1, -1,  1,  1,  0],
         [ 1,  1,  1,  0,  1],
         [ 1,  1,  1,  0, -1],
         [ 1,  1, -1,  0,  1],
         [ 1,  1, -1,  0, -1],
         [ 1, -1,  1,  0,  1],
         [ 1, -1,  1,  0, -1],
         [ 1, -1, -1,  0,  1],
         [ 1, -1, -1,  0, -1],
         [-1,  1, -1,  0,  1],
         [-1, -1,  1,  0, -1],
         [ 1,  1,  0,  1,  1],
         [ 1,  1,  0,  1, -1],
         [ 1,  1,  0, -1,  1],
         [ 1,  1,  0, -1, -1],
         [ 1, -1,  0,  1,  1],
         [ 1, -1,  0,  1, -1],
         [ 1, -1,  0, -1,  1],
         [ 1, -1,  0, -1, -1],
         [-1,  1,  0, -1,  1],
         [-1, -1,  0,  1, -1],
         [ 1,  0,  1,  1,  1],
         [ 1,  0,  1,  1, -1],
         [ 1,  0,  1, -1,  1],
        

 
i = 5
Si_shape: (32, 5)
Finished writing S5 to disk as S5.dat w/ 1.6e-07 GB
Finished writing X5 to disk as X5.dat w/ 3.84e-07 GB
Finished identifying 12 pfvs with non-empty extensions.
Finished writing S5_bar to disk as S5_bar.dat w/ 6e-08 GB
Sparsity of S5_bar: 0.0
Deleting S5 from disk, freeing 1.6e-07 GB
S5 deleted
Finished writing X5_bar to disk as X5_bar.dat w/ 1.44e-07 GB
Deleting X5 from disk, freeing 3.84e-07 GB
X5 deleted
Sparsity of X5_bar: 0.9166666666666666
Creating sparse version of X5_bar
Saving as X5_bar.sparse
Saved, using 2.04e-07 GB
Deleting X5_bar.dat, saving 1.44e-07 GB


(memmap([[ 1,  1,  1,  1,  1],
         [ 1,  1,  1, -1,  1],
         [ 1,  1,  1, -1, -1],
         [ 1,  1, -1,  1,  1],
         [ 1,  1, -1,  1, -1],
         [ 1, -1,  1,  1, -1],
         [ 1, -1,  1, -1,  1],
         [ 1, -1, -1,  1,  1],
         [ 1, -1, -1,  1, -1],
         [ 1, -1, -1, -1, -1],
         [-1,  1, -1, -1,  1],
         [-1, -1,  1,  1, -1]], dtype=int8),
 <COO: shape=(12, 12), dtype=int8, nnz=12, fill_value=0>)